In [1]:
with open('C:/Users/shameer/Desktop/New folder (4)/seq2seqBot/mu.txt', 'r', encoding = 'utf-8') as f:
  lines = f.read().split('\n')

In [2]:
lines[2]
len(lines)

873

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
num_samples = 10000
num_samples

10000

In [5]:
for line in lines[: min(num_samples, len(lines) - 1)]:
  line = line.lower()
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [6]:
input_texts[1]

'how are you doing ?'

In [7]:
target_texts[1]

'\tdoing good , thank you .\n'

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 872
Number of unique input tokens: 41
Number of unique output tokens: 51
Max sequence length for inputs: 124
Max sequence length for outputs: 223


In [9]:
print(input_characters)

[' ', '!', "'", ')', ',', '-', '.', '0', '1', '6', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
print(target_characters)

['\t', '\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [11]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [12]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [13]:
encoder_input_data.shape

(872, 124, 41)

In [14]:
decoder_input_data.shape

(872, 223, 51)

In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [16]:
encoder_input_data[1].shape

(124, 41)

In [17]:
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Users\shameer\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
batch_size = 64  # batch size for training
epochs = 3000  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [19]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [20]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 41)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 305152      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  315392      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [23]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('C:/Users/shameer/Desktop/New folder (4)/seq2seqBot/mytest/s2sb.h5')

Train on 697 samples, validate on 175 samples
Epoch 1/3000
697/697 [==============================] - 16s 23ms/step - loss: 0.9143 - val_loss: 0.6392
Epoch 2/3000
697/697 [==============================] - 16s 24ms/step - loss: 0.8174 - val_loss: 0.6367
Epoch 3/3000
697/697 [==============================] - 17s 24ms/step - loss: 0.8158 - val_loss: 0.6321
Epoch 4/3000
697/697 [==============================] - 17s 25ms/step - loss: 0.8119 - val_loss: 0.6289
Epoch 5/3000
697/697 [==============================] - 18s 26ms/step - loss: 0.8082 - val_loss: 0.6210
Epoch 6/3000
697/697 [==============================] - 19s 27ms/step - loss: 0.8013 - val_loss: 0.6172
Epoch 7/3000
697/697 [==============================] - 19s 27ms/step - loss: 0.7936 - val_loss: 0.6101
Epoch 8/3000
697/697 [==============================] - 19s 28ms/step - loss: 0.7831 - val_loss: 0.6033
Epoch 9/3000
697/697 [==============================] - 20s 29ms/step - loss: 0.7726 - val_loss: 0.5936
Epoch 10/3000
697/

697/697 [==============================] - 29s 42ms/step - loss: 0.4277 - val_loss: 0.4044
Epoch 79/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.4226 - val_loss: 0.4064
Epoch 80/3000
697/697 [==============================] - 30s 44ms/step - loss: 0.4198 - val_loss: 0.4130
Epoch 81/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.4178 - val_loss: 0.4057
Epoch 82/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.4132 - val_loss: 0.4071
Epoch 83/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.4101 - val_loss: 0.4062
Epoch 84/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.4055 - val_loss: 0.4076
Epoch 85/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.4017 - val_loss: 0.4070
Epoch 86/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.3970 - val_loss: 0.4075
Epoch 87/3000
697/697 [==============================] - 29s 42ms/ste

697/697 [==============================] - 29s 42ms/step - loss: 0.1920 - val_loss: 0.5136
Epoch 156/3000
697/697 [==============================] - 30s 44ms/step - loss: 0.1902 - val_loss: 0.5300
Epoch 157/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.1905 - val_loss: 0.5205
Epoch 158/3000
697/697 [==============================] - 30s 44ms/step - loss: 0.1876 - val_loss: 0.5156
Epoch 159/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.1845 - val_loss: 0.5195
Epoch 160/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.1801 - val_loss: 0.5247
Epoch 161/3000
697/697 [==============================] - 29s 42ms/step - loss: 0.1837 - val_loss: 0.5263
Epoch 162/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.1794 - val_loss: 0.5293
Epoch 163/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.1738 - val_loss: 0.5313
Epoch 164/3000
697/697 [==============================] - 29s

697/697 [==============================] - 31s 44ms/step - loss: 0.0824 - val_loss: 0.6774
Epoch 233/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0795 - val_loss: 0.6880
Epoch 234/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0820 - val_loss: 0.6817
Epoch 235/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0776 - val_loss: 0.6832
Epoch 236/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.0780 - val_loss: 0.6875
Epoch 237/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0767 - val_loss: 0.6926
Epoch 238/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0753 - val_loss: 0.6945
Epoch 239/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.0807 - val_loss: 0.6893
Epoch 240/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0729 - val_loss: 0.6927
Epoch 241/3000
697/697 [==============================] - 30s

697/697 [==============================] - 30s 43ms/step - loss: 0.0404 - val_loss: 0.8098
Epoch 310/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.0415 - val_loss: 0.8173
Epoch 311/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0513 - val_loss: 0.8096
Epoch 312/3000
697/697 [==============================] - 31s 44ms/step - loss: 0.0417 - val_loss: 0.8105
Epoch 313/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0420 - val_loss: 0.8093
Epoch 314/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0593 - val_loss: 0.8061
Epoch 315/3000
697/697 [==============================] - 30s 44ms/step - loss: 0.0413 - val_loss: 0.8119
Epoch 316/3000
697/697 [==============================] - 30s 42ms/step - loss: 0.0391 - val_loss: 0.8199
Epoch 317/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0518 - val_loss: 0.8177
Epoch 318/3000
697/697 [==============================] - 31s

...

697/697 [==============================] - 30s 43ms/step - loss: 9.3713e-04 - val_loss: 1.2800
Epoch 2737/3000
697/697 [==============================] - 30s 43ms/step - loss: 5.3550e-04 - val_loss: 1.2776
Epoch 2738/3000
697/697 [==============================] - 30s 43ms/step - loss: 7.8210e-04 - val_loss: 1.2812
Epoch 2739/3000
697/697 [==============================] - 30s 43ms/step - loss: 7.2497e-04 - val_loss: 1.2803
Epoch 2740/3000
697/697 [==============================] - 30s 44ms/step - loss: 5.9253e-04 - val_loss: 1.2750
Epoch 2741/3000
697/697 [==============================] - 30s 43ms/step - loss: 3.3472e-04 - val_loss: 1.2785
Epoch 2742/3000
697/697 [==============================] - 30s 43ms/step - loss: 1.2731e-04 - val_loss: 1.2739
Epoch 2743/3000
697/697 [==============================] - 30s 43ms/step - loss: 2.7437e-04 - val_loss: 1.2769
Epoch 2744/3000
697/697 [==============================] - 29s 42ms/step - loss: 0.0012 - val_loss: 1.2793
Epoch 2745/3000
697/6

697/697 [==============================] - 30s 43ms/step - loss: 0.0010 - val_loss: 1.2820
Epoch 2811/3000
697/697 [==============================] - 30s 44ms/step - loss: 9.0473e-04 - val_loss: 1.2770
Epoch 2812/3000
697/697 [==============================] - 29s 42ms/step - loss: 0.0015 - val_loss: 1.2778
Epoch 2813/3000
697/697 [==============================] - 30s 42ms/step - loss: 0.0012 - val_loss: 1.2712
Epoch 2814/3000
697/697 [==============================] - 31s 44ms/step - loss: 9.5136e-04 - val_loss: 1.2670
Epoch 2815/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0013 - val_loss: 1.2705
Epoch 2816/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0011 - val_loss: 1.2709
Epoch 2817/3000
697/697 [==============================] - 29s 42ms/step - loss: 9.7779e-04 - val_loss: 1.2757
Epoch 2818/3000
697/697 [==============================] - 30s 43ms/step - loss: 7.1402e-04 - val_loss: 1.2800
Epoch 2819/3000
697/697 [============

697/697 [==============================] - 30s 44ms/step - loss: 4.7504e-04 - val_loss: 1.2742
Epoch 2885/3000
697/697 [==============================] - 31s 44ms/step - loss: 4.7509e-04 - val_loss: 1.2823
Epoch 2886/3000
697/697 [==============================] - 30s 43ms/step - loss: 5.9719e-04 - val_loss: 1.2707
Epoch 2887/3000
697/697 [==============================] - 30s 43ms/step - loss: 6.9218e-04 - val_loss: 1.2763
Epoch 2888/3000
697/697 [==============================] - 31s 44ms/step - loss: 6.5016e-04 - val_loss: 1.2770
Epoch 2889/3000
697/697 [==============================] - 30s 43ms/step - loss: 2.4458e-04 - val_loss: 1.2751
Epoch 2890/3000
697/697 [==============================] - 30s 43ms/step - loss: 4.4581e-04 - val_loss: 1.2724
Epoch 2891/3000
697/697 [==============================] - 30s 43ms/step - loss: 5.2435e-04 - val_loss: 1.2777
Epoch 2892/3000
697/697 [==============================] - 30s 42ms/step - loss: 0.0016 - val_loss: 1.2764
Epoch 2893/3000
697/6

697/697 [==============================] - 29s 42ms/step - loss: 7.6513e-04 - val_loss: 1.2768
Epoch 2959/3000
697/697 [==============================] - 30s 42ms/step - loss: 5.8688e-04 - val_loss: 1.2810
Epoch 2960/3000
697/697 [==============================] - 29s 42ms/step - loss: 4.2415e-04 - val_loss: 1.2832
Epoch 2961/3000
697/697 [==============================] - 30s 43ms/step - loss: 8.9236e-04 - val_loss: 1.2848
Epoch 2962/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0011 - val_loss: 1.2772
Epoch 2963/3000
697/697 [==============================] - 30s 43ms/step - loss: 5.2604e-04 - val_loss: 1.2744
Epoch 2964/3000
697/697 [==============================] - 30s 43ms/step - loss: 3.8132e-04 - val_loss: 1.2745
Epoch 2965/3000
697/697 [==============================] - 31s 44ms/step - loss: 5.8995e-04 - val_loss: 1.2866
Epoch 2966/3000
697/697 [==============================] - 30s 43ms/step - loss: 0.0011 - val_loss: 1.2823
Epoch 2967/3000
697/697 [

C:\Users\shameer\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
